#hugging face 파일 다운로드 (서버로) 

cd /SAMPart3D/assets

wget https://huggingface.co/datasets/koyaniya/sampart3d-knight-table/resolve/main/table_colored.glb

#render file 

cd /SAMPart3D/tools

/root/blender-4.0.0-linux-x64/blender \
  -b \
  -P blender_render_16views.py \
  /SAMPart3D/assets/table_colored.glb \
  glb \
  /SAMPart3D/output/table_colored

In [1]:
import open3d as o3d
import numpy as np
import os
import trimesh
from collections import defaultdict

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
os.getcwd()

'/home/piai/SAMPart3D'

In [18]:
pcd = o3d.io.read_point_cloud("desk_full(1).ply")

points = np.asarray(pcd.points)
print("Total points:", points.shape)

Total points: (198400, 3)


In [4]:
mesh = o3d.io.read_triangle_mesh("/home/piai/source.glb")
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

## glb에 color 추가

In [9]:
def add_vertex_colors_to_scene(input_path, output_path):
    scene = trimesh.load(input_path, force='scene')
    
    if not isinstance(scene, trimesh.Scene):
        print(f"Error: Expected a trimesh.Scene but got {type(scene)}. Exiting.")
        return

    new_geometry = {}

    for name, mesh in scene.geometry.items():
        if isinstance(mesh, trimesh.Trimesh):
            print(f"\nProcessing mesh: {name}")

            
            num_vertices = len(mesh.vertices)
            
            white_colors = np.ones((num_vertices, 4), dtype=np.uint8) * 255 
            
            new_visual = trimesh.visual.ColorVisuals(
                mesh=mesh, 
                vertex_colors=white_colors
            )
            
            mesh.visual = new_visual
            print(f"Replaced mesh visual with ColorVisuals containing {num_vertices} white colors.")
            
            new_geometry[name] = mesh
        else:
            new_geometry[name] = mesh
            print(f"Skipping non-Trimesh geometry: {name} ({type(mesh)})")

    
    colored_scene = trimesh.Scene(geometry=new_geometry)
    
    
    colored_scene.export(output_path, file_type='glb')
    print(f"\nSuccessfully exported scene with vertex colors to: {output_path}")

input_glb = "source.glb"         
output_glb = "table_colored.glb"

add_vertex_colors_to_scene(input_glb, output_glb)


Processing mesh: Cube_OldTable_Material_0
Replaced mesh visual with ColorVisuals containing 2318 white colors.

Successfully exported scene with vertex colors to: table_colored.glb


In [4]:
mesh = o3d.io.read_triangle_mesh("/home/piai/SAMPart3D/assets/source.glb")

# 🔎 sanity check
assert len(mesh.vertices) > 0, "Mesh has no vertices"
assert len(mesh.triangles) > 0, "Mesh has no triangles"

# (권장) 노멀 계산
mesh.compute_vertex_normals()

# 모든 vertex를 빨간색으로
colors = np.ones((len(mesh.vertices), 3))
colors[:, 0] = 1.0  # R
colors[:, 1] = 0.0  # G
colors[:, 2] = 0.0  # B

mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

# GLB 저장 (옵션 중요)
o3d.io.write_triangle_mesh(
    "desk_colored.glb",
    mesh,
    write_vertex_colors=True,
    write_triangle_uvs=False
)


True

In [12]:
mesh = o3d.io.read_triangle_mesh("table_colored.glb")
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

In [11]:
mesh = trimesh.load("table_colored.glb", force='mesh')

print("Has vertex colors:", hasattr(mesh.visual, "vertex_colors"))
if hasattr(mesh.visual, "vertex_colors"):
    vc = mesh.visual.vertex_colors
    print("Vertex colors shape:", None if vc is None else vc.shape)

Has vertex colors: True
Vertex colors shape: (2318, 4)


In [19]:
o3d.visualization.draw_geometries([pcd])

In [20]:
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1.5, max_nn=30))
pcd.orient_normals_consistent_tangent_plane(k=30)

# 3. Create the mesh using Ball Pivoting Algorithm (BPA)
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 1.5 * avg_dist  # Tune this radius based on your data density

mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector([radius, radius * 2])
)

# NEW CHECK HERE:
print(f"Number of vertices in the mesh: {len(mesh.vertices)}")
print(f"Number of triangles in the mesh: {len(mesh.triangles)}")
# If the number of triangles is 0, the reconstruction failed.

# 4. Save the resulting mesh as a GLB file
output_file = "reconstructed_mesh.glb"
o3d.io.write_triangle_mesh(output_file, mesh)

print(f"Mesh successfully saved to {output_file}")

Number of vertices in the mesh: 198400
Number of triangles in the mesh: 260754
Mesh successfully saved to reconstructed_mesh.glb


In [25]:
mesh = o3d.io.read_triangle_mesh("/home/piai/dust3r_project/PartNeXt/PartNeXt_mesh/glbs/000-134/87e55e5e08b04d74957b771f9b5412b3.glb")
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

In [7]:
import os 

In [8]:
os.getcwd()

'/home/piai'

In [13]:
output_file = "reconstructed_mesh.glb"

# Check if the file exists before attempting to load
if os.path.exists(output_file):
    # Load the GLB file as a TriangleMesh object
    loaded_mesh = o3d.io.read_triangle_mesh(output_file)

    # Visualize the mesh
    print(f"Opening visualization for {output_file}...")
    o3d.visualization.draw_geometries([loaded_mesh])
else:
    print(f"Error: The file '{output_file}' was not found. Please ensure the reconstruction step ran successfully.")

[Open3D WARNING] Unable to load file reconstructed_mesh.glb with ASSIMP
Opening visualization for reconstructed_mesh.glb...
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


## PLY & NPY 통합

In [37]:
data = np.load("mesh_0.5.npy")
print(type(data))
print(data.shape)
print(data.dtype)

<class 'numpy.ndarray'>
(1574,)
int64


In [52]:
labels = np.load("mesh_1.5.npy")
print("Unique part IDs:", np.unique(labels))
print("Number of parts:", len(np.unique(labels)))

Unique part IDs: [0 1 2 3 4 5 6 7 8]
Number of parts: 9


In [53]:
print("Labels:", len(labels))

Labels: 1574


In [27]:
for pid in np.unique(labels):
    print(f"Part {pid}: {(labels == pid).sum()} faces")

Part 0: 4 faces
Part 1: 12 faces
Part 2: 60 faces
Part 3: 508 faces
Part 4: 109 faces
Part 5: 22 faces
Part 6: 19 faces
Part 7: 10 faces
Part 8: 830 faces


In [81]:
knight = o3d.io.read_point_cloud("1.5.ply")

points = np.asarray(knight.points)
colors = np.asarray(knight.colors)
print("Total points:", points.shape)
o3d.visualization.draw_geometries([knight])


colors_q = np.round(colors * 255).astype(int)

groups = defaultdict(list)
for i, c in enumerate(map(tuple, colors_q)):
    groups[c].append(i)

print("Number of color groups:", len(groups))

Total points: (13741, 3)
Number of color groups: 9


In [75]:
knight_mesh = o3d.io.read_triangle_mesh("1.5.ply")
knight_mesh.compute_vertex_normals()
triangles = np.asarray(knight_mesh.triangles)
vertices = np.asarray(knight_mesh.vertices)
print("Vertices:", vertices.shape)
print("Triangles:", triangles.shape)
print("Labels:", labels.shape)

[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).
Vertices: (13741, 3)
Triangles: (0, 3)
Labels: (1574,)


## Color별 분리

In [62]:
knight = o3d.io.read_point_cloud("1.5.ply")

points = np.asarray(knight.points)
colors = np.asarray(knight.colors)

print("Points:", points.shape)
print("Colors:", colors.shape)

colors_rounded = np.round(colors, decimals=4)

unique_colors, inverse_idx = np.unique(
    colors_rounded, axis=0, return_inverse=True
)

print("Number of parts (unique colors):", len(unique_colors))

Points: (13741, 3)
Colors: (13741, 3)
Number of parts (unique colors): 9


In [63]:
out_dir = "color_parts"
os.makedirs(out_dir, exist_ok=True)

for part_id, color in enumerate(unique_colors):
    idx = inverse_idx == part_id

    part_points = points[idx]
    part_colors = colors[idx]

    part_pcd = o3d.geometry.PointCloud()
    part_pcd.points = o3d.utility.Vector3dVector(part_points)
    part_pcd.colors = o3d.utility.Vector3dVector(part_colors)

    out_path = f"{out_dir}/part_{part_id}.ply"
    o3d.io.write_point_cloud(out_path, part_pcd)

    print(f"Saved {out_path} → {part_points.shape[0]} points")

Saved color_parts/part_0.ply → 136 points
Saved color_parts/part_1.ply → 75 points
Saved color_parts/part_2.ply → 844 points
Saved color_parts/part_3.ply → 65 points
Saved color_parts/part_4.ply → 679 points
Saved color_parts/part_5.ply → 319 points
Saved color_parts/part_6.ply → 2249 points
Saved color_parts/part_7.ply → 8365 points
Saved color_parts/part_8.ply → 1009 points


In [85]:
part = o3d.io.read_point_cloud("color_parts/part_7.ply")
o3d.visualization.draw_geometries([part])